In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/titanic.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

In [5]:
data.columns

In [6]:
data.select("Name", "Age","Survived").show()

In [7]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [8]:
my_cols.show()

In [9]:
my_final_data = my_cols.na.drop()

In [10]:
my_final_data.show()

In [11]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [12]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_indexed = gender_indexer.fit(my_final_data).transform(my_final_data)

gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')
gender_encoded = gender_encoder.transform(gender_indexed) 

In [13]:
gender_encoded.show()


In [14]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_indexed = embark_indexer.fit(gender_encoded).transform(gender_encoded)

embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')
embark_encoded = embark_encoder.transform(embark_indexed) 

In [15]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

assembled = assembler.transform(embark_encoded)
assembled.show()

In [16]:
assembled.select("features").show()

In [17]:
from pyspark.ml.classification import LogisticRegression


In [18]:
train_titanic_data, test_titanic_data = assembled.randomSplit([0.7,.3])

In [19]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')
lr_fitted = log_reg_titanic.fit(train_titanic_data)

In [20]:
results = lr_fitted.transform(test_titanic_data)
results.select("Survived","prediction").show()

In [21]:
results.select('Survived','prediction').show()

In [22]:
results.createTempView("titanic_lr")

In [23]:
%sql
select count(*) from titanic_lr where Survived=prediction